In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import mxnet as mx
from mxnet import nd
import mxnet.gluon as gl
import numpy as np

In [3]:
test_root = './data/ai_challenger_scene_test_a_20170922/'

In [4]:
def transform_test(img):
    if min(img.shape[0], img.shape[1]) < 224:
        img = mx.image.resize_short(img, 224)
    im_h, im_w = img.shape[0], img.shape[1]
    img0 = mx.image.center_crop(img, (224, 224))[0]
    img1 = mx.image.fixed_crop(img, 0, 0, 224, 224)
    img2 = mx.image.fixed_crop(img, im_w-224, 0, 224, 224)
    img3 = mx.image.fixed_crop(img, 0, im_h-224, 224, 224)
    img4 = mx.image.fixed_crop(img, im_w-224, im_h-224, 224, 224)
    
    img0 = img0.astype('float32') / 255
    img1 = img1.astype('float32') / 255
    img2 = img2.astype('float32') / 255
    img3 = img3.astype('float32') / 255
    img4 = img4.astype('float32') / 255
    
    img0 = mx.image.color_normalize(img0, mean=nd.array([0.4960, 0.4781, 0.4477]), 
                             std=nd.array([0.2915, 0.2864, 0.2981]))
    img1 = mx.image.color_normalize(img1, mean=nd.array([0.4960, 0.4781, 0.4477]), 
                             std=nd.array([0.2915, 0.2864, 0.2981]))
    img2 = mx.image.color_normalize(img2, mean=nd.array([0.4960, 0.4781, 0.4477]), 
                             std=nd.array([0.2915, 0.2864, 0.2981]))
    img3 = mx.image.color_normalize(img3, mean=nd.array([0.4960, 0.4781, 0.4477]), 
                             std=nd.array([0.2915, 0.2864, 0.2981]))
    img4 = mx.image.color_normalize(img4, mean=nd.array([0.4960, 0.4781, 0.4477]), 
                             std=nd.array([0.2915, 0.2864, 0.2981]))
    
    img0 = nd.transpose(img0, (2, 0, 1))
    img1 = nd.transpose(img1, (2, 0, 1))
    img2 = nd.transpose(img2, (2, 0, 1))
    img3 = nd.transpose(img3, (2, 0, 1))
    img4 = nd.transpose(img4, (2, 0, 1))
    
    return img0, img1, img2, img3, img4

In [5]:
from data_utils import TestDataSet
from dataloader import DataLoader

In [6]:
test_img_path = test_root + '/scene_test_a_images_20170922/'
test_set = TestDataSet(test_img_path, transform_test)
test_data = DataLoader(test_set, 32, shuffle=False, last_batch='keep')

In [7]:
ctx = mx.gpu(0)
net = gl.model_zoo.vision.resnet50_v2(classes=80)
# net.initialize(ctx=ctx)
net.load_params('./net.params', ctx=ctx)
net.hybridize()

In [8]:
import json

In [9]:
sub_list = []
i = 0
for im_name, im0, im1, im2, im3, im4 in test_data:
    im0 = im0.as_in_context(ctx)
    im1 = im1.as_in_context(ctx)
    im2 = im2.as_in_context(ctx)
    im3 = im3.as_in_context(ctx)
    im4 = im4.as_in_context(ctx)
    
    out0 = net(im0)
    out1 = net(im1)
    out2 = net(im2)
    out3 = net(im3)
    out4 = net(im4)
    
    out = (out0 + out1 + out2 + out3 + out4) / 5
    pred = nd.topk(out, 1, k=3)
    
    for n in range(len(im_name)):
        temp_dict = {'image_id': im_name[n], 'label_id': [int(i) for i in pred[n].asnumpy()]}
        sub_list.append(temp_dict)

In [10]:
with open('submission.json', 'w') as fout:
    json.dump(sub_list, fout)